In [123]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os
from sklearn.cluster import KMeans

Populating the interactive namespace from numpy and matplotlib


In [82]:
hdfsclient = Client("amp-spark-master.amp", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/tmp/log"

In [139]:
executor_mem = "200g"
cores_per_executor = 32
num_executors = 14
EGR1_PATH = '/user/vaishaal/endive-data/EGR1'

In [ ]:
# this is where the magic happens
results = [] 
num_filters = 256
gamma = 1.0
kmer_size=8
res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           "/tmp/filters.csv", 
                           logpath, 
                           seed=0,
                           filter_gen_gen=make_gaussian_filter_gen,
                           cores_per_executor=cores_per_executor,
                           num_filters=num_filters,
                           sample=1.0,
                           alphabet_size=4,
                           kmer_size=kmer_size,
                           gamma=gamma,
                           executor_mem=executor_mem,
                           num_partitions=cores_per_executor*num_executors,
                           featuresOutput="/user/vaishaal/featurizedWindows/sequence/full/{0}-filters/{1}-gamma/{2}-kmers/replicate-0".format(num_filters, gamma, kmer_size),
                           num_executors=num_executors)
results.append(res)

ALPHABET SIZE  4
(256, 32)
{'kmerLength': 8, 'reference': '/home/eecs/akmorrow/ADAM/endive/workfiles/hg19.2bit', 'filtersPath': '/tmp/filters.csv', 'numPartitions': 448, 'dnaseBams': '/data/anv/DREAMDATA/dnase_bams/merged_coverage/', 'aggregatedSequenceOutput': '/user/vaishaal/endive-data/EGR1', 'featuresOutput': '/user/vaishaal/featurizedWindows/sequence/full/256-filters/1.0-gamma/8-kmers/replicate-0', 'seed': 0, 'dnaseNarrow': '/data/anv/DREAMDATA/DNASE/peaks/relaxed/', 'approxDim': 256, 'featurizeSample': 1.0, 'alphabetSize': 4, 'readFiltersFromDisk': True}


In [62]:
chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])

In [ ]:
results_df = cross_validate("/user/vaishaal/featurizedWindows/sequence/tiny/{0}-filters/{1}-gamma/{2}-kmers/replicate-0".format(num_filters, gamma, kmer_size), 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            numHoldOutChr=1, 
                            numHoldOutCell=1,
                            cores_per_executor = 32,
                            num_executors = 14,
                            executor_mem="150g",
                            num_folds=1, 
                            negativeSamplingFreqs=[1.0],
                            regs=[1e-8])
    

In [68]:
old = results_df

In [130]:
results_df

,negativeSamplingFreq,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,1,1.000000e-08,[3],[chr12],0.006357,0.83125,0,0,0,0,0.002056,0.854578,0,0,0,0


In [129]:
print("POOP")

POOP
